In [28]:
import sys

sys.path.insert(0, '..')
import geopandas
import maup
from read_config import read_yaml
import shapely
import pandas as pd

In [4]:
config = read_yaml('../preprocessing_config.yaml')
districts_geo_path = config['ARKANSAS']['DISTRICTS_GEO_PATH']
precinct_election_path = config['ARKANSAS']['PRECINCTS_ELECTION_PATH']

In [12]:
precincts_election = geopandas.read_file(precinct_election_path)

In [16]:
precincts = precincts_election[['COUNTY_FIP', 'PRECINCT', 'geometry']]
precincts.rename(columns={'COUNTY_FIP': 'countyId', 'PRECINCT': 'precinctName'}, inplace=True)

/Users/kostadindev/.conda/envs/preprocessing/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [17]:
precincts

,countyId,precinctId,geometry
0,037,Bedford,"POLYGON ((-90.82801 35.21995, -90.83057 35.219..."
1,143,War Eagle,"POLYGON ((-94.01246 36.21703, -94.01248 36.216..."
2,037,Smith,"POLYGON ((-90.61177 35.28176, -90.61193 35.267..."
3,143,Dutch Mills,"POLYGON ((-94.45759 35.90149, -94.45766 35.897..."
4,131,9-4J,"POLYGON ((-94.28547 35.22252, -94.28546 35.222..."
...,...,...,...
2520,065,Sage,"MULTIPOLYGON (((-91.83684 36.04460, -91.83677 ..."
2521,015,Dry Fork,"POLYGON ((-93.44279 36.17523, -93.44303 36.173..."
2522,123,Colt City/Country/Pine Tree,"MULTIPOLYGON (((-90.88072 35.05231, -90.88024 ..."
2523,123,Forrest City Ward 3,"POLYGON ((-90.79779 35.02263, -90.79779 35.022..."


In [31]:
def get_polygons(geometry):
    polygons = []
    if type(geometry) == shapely.geometry.MultiPolygon:
        for shape in geometry:
            polygons.extend(get_polygons(shape))
    elif type(geometry) == shapely.geometry.Polygon:
        polygons.append(geometry)
    return polygons


def fix_geometry(precincts):
    rows = precincts.values
    processed_precincts = []
    id = 0
    for row in rows[:100]:
        countyId = row[0]
        precinctName = row[1]
        polygons = get_polygons(row[2])
        for polygon in polygons:
            processed_precincts.append([countyId, precinctName, id, polygon])
            id += 1
    processed_precincts = geopandas.GeoDataFrame(pd.DataFrame(processed_precincts, columns= ['countyId', 'precinctName', 'precinctId', 'geometry']))
    processed_precincts.crs = precincts.crs
    return processed_precincts

In [34]:
precincts = fix_geometry(precincts)

/var/folders/25/vw8n1j616yz1zwrd2h0rc8100000gn/T/ipykernel_8038/1135342073.py:4: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for shape in geometry:


In [35]:
precincts.head()

,countyId,precinctName,precinctId,geometry
0,037,Bedford,0,"POLYGON ((-90.82801 35.21995, -90.83057 35.219..."
1,143,War Eagle,1,"POLYGON ((-94.01246 36.21703, -94.01248 36.216..."
2,037,Smith,2,"POLYGON ((-90.61177 35.28176, -90.61193 35.267..."
3,143,Dutch Mills,3,"POLYGON ((-94.45759 35.90149, -94.45766 35.897..."
4,131,9-4J,4,"POLYGON ((-94.28547 35.22252, -94.28546 35.222..."


In [ ]:
precincts_path = config['ARKANSAS']['PRECINCTS_PATH']
precincts.to_file(precincts_path)

